In [1]:
import numpy as np
import pandas as pd
import datetime
import random

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
pd.set_option('display.max_columns', None)

In [21]:
X = pd.read_csv('./dataset/X.csv')
df_test  = pd.read_csv('./dataset/X_test.csv')
X.shape, df_test.shape

((32820, 531), (14067, 531))

In [22]:
y=pd.read_csv('./dataset/df_labels.csv')
y.shape

(32820, 1)

In [23]:
df_test[np.isnan(df_test)]=0

In [24]:
# scaler_x = MinMaxScaler()
# scaler_y = MinMaxScaler()
# print(scaler_x.fit(X))
# xscale=scaler_x.transform(X)
# print(scaler_y.fit(y))
# yscale=scaler_y.transform(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
def build_model():
    model = Sequential([
    Dense(64, activation=tf.nn.relu, input_shape=[len(X_train.keys())]),
    Dense(64, activation=tf.nn.relu),         
    Dense(1)
  ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.0001)
    
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse','msle'])
    return model

In [27]:
regressor = KerasRegressor(build_fn=build_model, batch_size=32, epochs=50)

In [28]:
results = regressor.fit(X_train,y_train)

Epoch 1/50
22974/22974 [==============================] - 3s 121us/sample - loss: 12.4341 - mean_squared_error: 12.4341 - mean_squared_logarithmic_error: 0.4433
Epoch 2/50
22974/22974 [==============================] - 3s 115us/sample - loss: 1.6279 - mean_squared_error: 1.6279 - mean_squared_logarithmic_error: 0.0239
Epoch 3/50
22974/22974 [==============================] - 3s 114us/sample - loss: 1.4628 - mean_squared_error: 1.4628 - mean_squared_logarithmic_error: 0.0217
Epoch 4/50
22974/22974 [==============================] - 3s 114us/sample - loss: 1.4319 - mean_squared_error: 1.4319 - mean_squared_logarithmic_error: 0.0213
Epoch 5/50
22974/22974 [==============================] - 3s 115us/sample - loss: 1.4142 - mean_squared_error: 1.4142 - mean_squared_logarithmic_error: 0.0211
Epoch 6/50
22974/22974 [==============================] - 3s 115us/sample - loss: 1.4036 - mean_squared_error: 1.4036 - mean_squared_logarithmic_error: 0.0209
Epoch 7/50
22974/22974 [====================

In [29]:
from sklearn.metrics import mean_squared_log_error

pred_train= regressor.predict(X_train)
print(mean_squared_log_error(y_train,pred_train)*100)

pred_test= regressor.predict(X_test)
print(mean_squared_log_error(y_test,pred_test)*100)

2.027307256289269
2.108539500034366


In [30]:
# Read in sample_submission dataframe
submission = pd.read_csv("./dataset/sample_submission_iwBpW0t.csv")
submission.shape

(14067, 2)

In [31]:
y_pred = regressor.predict(df_test)

In [32]:
# Append predictions from blended models
submission.iloc[:,1] = np.floor(np.expm1(y_pred))

In [33]:
# Fix outleir predictions
q1 = submission['cc_cons'].quantile(0.0001)
q2 = submission['cc_cons'].quantile(0.98)
submission['cc_cons'] = submission['cc_cons'].apply(lambda x: x if x > q1 else x*0.77)
submission['cc_cons'] = submission['cc_cons'].apply(lambda x: x if x < q2 else x*1.1)
submission.to_csv("./dataset/submission_keras1.csv", index=False)

In [34]:
# Scale predictions
submission['cc_cons'] *= 1.001619
submission.to_csv("./dataset/submission_keras2.csv", index=False)